In [ ]:
import os, json
from dotenv import load_dotenv
from weather import WeatherAPI, format_current_weather
load_dotenv()

# AWS Credentials
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
print(ACCESS_KEY)

In [2]:
import boto3
import json
import time
from typing import List, Dict, Any, Optional

client_kwargs = {}
client_kwargs["aws_access_key_id"] = ACCESS_KEY
client_kwargs["aws_secret_access_key"] = SECRET_KEY

class BedrockLlamaAgent:
    """
    An agent implementation using Amazon Bedrock with Llama model.
    This agent can process user inputs, maintain context, and generate responses.
    """
    
    def __init__(
        self,
        model_id: str = "meta.llama3-3-70b-instruct-v1:0",
        region_name: str = "us-east-2",
        temperature: float = 0.7,
        max_tokens: int = 512,
        context_window_size: int = 4096
    ):
        """
        Initialize the Bedrock Llama Agent.
        
        Args:
            model_id: The Bedrock model ID for Llama
            region_name: AWS region where Bedrock is deployed
            temperature: Controls randomness in response generation
            max_tokens: Maximum token length for generated response
            context_window_size: Maximum context window size
        """
        self.bedrock_runtime = boto3.client(
            service_name="bedrock-runtime",
            region_name=region_name,
            **client_kwargs  # Pass AWS credentials
        )
        self.model_id = model_id
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.context_window_size = context_window_size
        self.conversation_history = []
        self.tools = []


In [3]:
class AgentTool:
    """Base class for defining tools that the agent can use."""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        
    def execute(self, *args, **kwargs):
        """Tool implementation to be overridden by subclasses."""
        raise NotImplementedError("Tool execution not implemented")


class WeatherTool(AgentTool):
    """Example weather lookup tool."""
    
    def __init__(self):
        super().__init__(
            name="get_weather",
            description="Get current weather information for a location"
        )
        
    def execute(self, location: str) -> Dict[str, Any]:
        """
        Get weather for the given location.
        
        Args:
            location: City or location name
            
        Returns:
            Weather information
        """
        weather_api = WeatherAPI(OPENWEATHER_API_KEY) #args.api_key)
        
        # Get and display current weather
        current_weather = weather_api.get_current_weather(location)
        return format_current_weather(current_weather)

In [4]:
messages = [
        {
            "role": "user",
            "content": [{"text":"What is weather in Atlanta today?"}]
        }
    ]

agent = BedrockLlamaAgent()

# Create and register tools
weather_tool = WeatherTool()
follow_up_content_blocks = []
toolConfig = {
    "tools": [
        {
            "toolSpec":{
    "name": weather_tool.name,
    "description": weather_tool.description,
    "inputSchema": {
        "json":{
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city or location to get weather for"
            }
        },
        "required": ["location"]
    }
    }
}
        }
    ]

}

In [5]:
def converse():
    # Initialize the agent    

    converse_api_params = {
        "modelId": agent.model_id,
        "toolConfig": toolConfig,
        "messages": messages,
        "inferenceConfig" : {
            "maxTokens": agent.max_tokens,
            "temperature": agent.temperature
        }
    }

    response = agent.bedrock_runtime.converse(**converse_api_params)
    print(response["output"]["message"])
    return response["output"]["message"]

In [6]:
def process_tool(response_content_blocks, follow_up_content_blocks):
    print(response_content_blocks['content'])

    for content_block in response_content_blocks['content']:
        if 'toolUse' in content_block:
            tool_use_block = content_block['toolUse']
            tool_use_name = tool_use_block['name']
            
            print(f"Using tool {tool_use_name}")
            
            if tool_use_name == 'get_weather':
                weather_data = weather_tool.execute(tool_use_block['input']['location'])
                print(weather_data)
                result = {
                    "location": tool_use_block['input']['location'],
                    "temperature": weather_data["temperature"],
                    "condition": weather_data["condition"],
                    "humidity": weather_data["humidity"],
                    "wind_speed": weather_data["wind_speed"]
                }
                follow_up_content_blocks.append({
                    "toolResult": {
                        "toolUseId": tool_use_block['toolUseId'],
                        "content": [
                            {
                                "json": {
                                    "result": result
                                }
                            }
                        ]
                    }
                })
        elif 'text' in content_block:
            print(content_block['text'])
    return follow_up_content_blocks

In [7]:
def process_request_after_tool(follow_up_content_blocks):
    if len(follow_up_content_blocks) > 0:
        
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        messages.append(follow_up_message)

        print(messages)
        converse_api_params = {
            "modelId": agent.model_id,
            "toolConfig": toolConfig,
            "messages": messages,
            "inferenceConfig" : {
                "maxTokens": agent.max_tokens,
                "temperature": agent.temperature
            }
        }

        response = agent.bedrock_runtime.converse(**converse_api_params)
        return response["output"]["message"]

In [ ]:
response_content_blocks = converse()

In [ ]:
messages.append(response_content_blocks)
follow_up_content_blocks = process_tool(response_content_blocks, follow_up_content_blocks)

In [ ]:
final_response = process_request_after_tool(follow_up_content_blocks)
print(f"output is {final_response}")

In [ ]:
messages = [
        {
            "role": "user",
            "content": [{"text":"What is weather in Atlanta today?"}]
        }
    ]

agent = BedrockLlamaAgent()

# Create and register tools
weather_tool = WeatherTool()
follow_up_content_blocks = []